In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sqlContext

In [2]:
sc.stop()

In [2]:
df = sqlContext.read.format('com.databricks.spark.csv')\
      .options(header='true', inferschema='true')\
      .load('/home/affine/Desktop/Affine/07PySpark/Assignment02/train.csv')
      
print(df)


DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]


In [3]:
df.show()
print(df.describe().select('Fare'))
df.printSchema()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [15]:
#1.Find number of people which are male and female.
print("Male Population  ",df.filter(df['Sex']=='male').count())
print("Female Population  " ,df.filter(df['Sex']=='female').count())


Male Population   577
Female Population   314


In [16]:
#2.Convert datatype of Fare class to double using udf
print(df.select('Fare'))

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def string_to_double(x):return double(x)
udfstring_to_double = udf(string_to_double, StringType())
df.withColumn("Fare",udfstring_to_double("Fare") )

print(df.select('Fare'))
df.select('Fare').show()
df.printSchema()

DataFrame[Fare: double]
DataFrame[Fare: double]
+-------+
|   Fare|
+-------+
|   7.25|
|71.2833|
|  7.925|
|   53.1|
|   8.05|
| 8.4583|
|51.8625|
| 21.075|
|11.1333|
|30.0708|
|   16.7|
|  26.55|
|   8.05|
| 31.275|
| 7.8542|
|   16.0|
| 29.125|
|   13.0|
|   18.0|
|  7.225|
+-------+
only showing top 20 rows

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
#3.Get average age of all the survived Passenger
from pyspark.sql.functions import mean, min, max
df.select('Age').filter(df['Survived']==1).select([mean('Age')]).show()



+------------------+
|          avg(Age)|
+------------------+
|28.343689655172415|
+------------------+



In [8]:
#4.Get Average Ticket Fare for different passenger class
#df.show()
df.groupBy('Pclass').mean('Fare').show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.15468749999992|
|     3|13.675550101832997|
|     2| 20.66218315217391|
+------+------------------+



In [155]:
#5.Count number of male people whose age is >=20 and <=50
df.filter((df['Age']>=20) & (df['Age']<=50) & (df['Sex']=='male')).count()

317

In [9]:
#Q6) Get Maximum Fare and average Age
df.select([max('Fare'),mean('Age')]).show()

+---------+-----------------+
|max(Fare)|         avg(Age)|
+---------+-----------------+
| 512.3292|29.69911764705882|
+---------+-----------------+



In [157]:
#Q7) Get Average age of all the males and females who were drowned.
df.filter(df['Survived']==0).select([mean('Age')]).show()

+-----------------+
|         avg(Age)|
+-----------------+
|30.62617924528302|
+-----------------+



In [197]:
df.show()
df.select('Age').collect()[2][0] * 1000

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

26000.0